### Подход 1: градиентный бустинг "в лоб"

Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.


Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [1]:
import pandas
import numpy as np
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score

seed = 42

In [2]:
features = pandas.read_csv('./data/features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


In [3]:
#Убираем признаки, отсутствующие в тестовой выборке:
X = features.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_dire',\
                   'barracks_status_radiant'],axis=1)


Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
#Смотрим на стобцы с пропусками:
c = X.count()
c[c.values<len(X)]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

**Почему могут быть пропуски в данных?**

Например, для *first_blood_time* и *dire_bottle_time*.

Скорее всего, из-за того, что событие не произошло в первые 5 минут матча. Для признака *first_blood_time* это явно указано в final_statement.

Замените пропуски на нули с помощью функции fillna().

Какой столбец содержит целевую переменную? Запишите его название.

In [5]:
#Заполняем пропуски:
X.fillna(0, inplace=True)

#Целевая переменная *radiant_win*:
y = features['radiant_win']

Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30).

In [14]:
trees = [10, 20, 30, 50, 100]
kf = KFold(len(X), n_folds=5, shuffle=True, random_state=seed)
cv_result = []
for t in trees:
    clf = GradientBoostingClassifier(n_estimators=t, random_state=seed)
    start_time = datetime.datetime.now()
    print 'Training for ', t
    scores = cross_val_score(clf, X, y, scoring='roc_auc', cv=kf)
    print 'Time elapsed:', datetime.datetime.now() - start_time
    cv_result.append(np.mean(scores))

Training for  10
Time elapsed: 0:00:29.259000
Training for  20
Time elapsed: 0:00:51.784000
Training for  30
Time elapsed: 0:01:18.581000
Training for  50
Time elapsed: 0:02:01.877000
Training for  100
Time elapsed: 0:04:20.090000


In [15]:
#Смотрим на результаты кросс-валидации:
zip(trees,cv_result)

[(10, 0.66485069327216473),
 (20, 0.68246188104131489),
 (30, 0.69000646786084108),
 (50, 0.6974943598879767),
 (100, 0.70622297734006678)]

### Отчет по градиентному бустингу:

*Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?*

Ответ: Например, для first_blood_time и dire_bottle_time.
Скорее всего, из-за того, что событие не произошло в первые 5 минут матча. Для признака first_blood_time это явно указано в final_statement.

*Как называется столбец, содержащий целевую переменную?*

Ответ: radiant_win

*Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось?*

Ответ: Кросс-валидация заняла 0:01:18.581000, качество по *auc_roc* составило 0.6900

*Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?*

Ответ: Да, имеет. Видно, что качество продолжает расти и после 30 деревьев. Для ускорения обучения можно использовать часть обучающей выборки (вместо целой), уменьшить число деревьев и/или их глубину. При повторном обучении классификатора можно использовать параметр *warm_start=True*. Если у процессора несколько ядер, для *cross_val_score* можно увеличить параметр n_jobs.


### Подход 2: логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорение анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C).

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import grid_search

def bestC(clf, X, y):
    #Функция для поиска оптимального С
    grid = {'C': np.power(10.0, np.arange(-10, 10))}
    gs = grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=kf)
    gs.fit(X, y)
    return gs.best_estimator_, gs.grid_scores_
    
kf = KFold(len(X), n_folds=5, shuffle=True, random_state=seed)
X_scaled = StandardScaler().fit_transform(X)
clfLog = LogisticRegression(random_state=seed)

clfLog, gsscores = bestC(clfLog, X_scaled, y)
gsscores


[mean: 0.58637, std: 0.01015, params: {'C': 1e-10},
 mean: 0.68715, std: 0.00125, params: {'C': 1.0000000000000001e-09},
 mean: 0.68716, std: 0.00125, params: {'C': 1e-08},
 mean: 0.68726, std: 0.00125, params: {'C': 9.9999999999999995e-08},
 mean: 0.68819, std: 0.00125, params: {'C': 9.9999999999999995e-07},
 mean: 0.69516, std: 0.00125, params: {'C': 1.0000000000000001e-05},
 mean: 0.71136, std: 0.00130, params: {'C': 0.0001},
 mean: 0.71636, std: 0.00127, params: {'C': 0.001},
 mean: 0.71655, std: 0.00124, params: {'C': 0.01},
 mean: 0.71653, std: 0.00124, params: {'C': 0.10000000000000001},
 mean: 0.71652, std: 0.00123, params: {'C': 1.0},
 mean: 0.71652, std: 0.00124, params: {'C': 10.0},
 mean: 0.71652, std: 0.00124, params: {'C': 100.0},
 mean: 0.71652, std: 0.00124, params: {'C': 1000.0},
 mean: 0.71652, std: 0.00124, params: {'C': 10000.0},
 mean: 0.71652, std: 0.00124, params: {'C': 100000.0},
 mean: 0.71652, std: 0.00124, params: {'C': 1000000.0},
 mean: 0.71652, std: 0.0012

In [9]:
#Выбираем наилучший параметр С и смотрим на время обучения логистической регрессии:
start_time = datetime.datetime.now()
print 'Training for the best C parameter = 0.01'
scores = cross_val_score(clfLog, X_scaled, y, scoring='roc_auc', cv=kf)
print 'Time elapsed:', datetime.datetime.now() - start_time, 'Score = ', np.mean(scores)

Training for the best C parameter = 0.01
Time elapsed: 0:00:15.376000 Score =  0.716550270785


Видим, что логистическая регрессия обучается значительно быстрее градиентного бустинга.

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальный признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборке, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [10]:
##Убираем категориальные признаки из выборки:
categor_cols =  []
for c in X.columns:
    if c.find('_hero')>0:
        categor_cols.append(c)
categor_cols.append('lobby_type')

X_nocat = X.drop(categor_cols, axis=1)

In [11]:
##Повторяем операцию с поиском наилучшего параметра для логистической регрессии:
X_nocat = StandardScaler().fit_transform(X_nocat)
clfLog = LogisticRegression(random_state=seed)

clfLog, gsscores = bestC(clfLog, X_nocat, y)
gsscores

[mean: 0.59476, std: 0.01403, params: {'C': 1e-10},
 mean: 0.68711, std: 0.00128, params: {'C': 1.0000000000000001e-09},
 mean: 0.68712, std: 0.00128, params: {'C': 1e-08},
 mean: 0.68721, std: 0.00128, params: {'C': 9.9999999999999995e-08},
 mean: 0.68814, std: 0.00128, params: {'C': 9.9999999999999995e-07},
 mean: 0.69510, std: 0.00130, params: {'C': 1.0000000000000001e-05},
 mean: 0.71134, std: 0.00140, params: {'C': 0.0001},
 mean: 0.71638, std: 0.00137, params: {'C': 0.001},
 mean: 0.71656, std: 0.00133, params: {'C': 0.01},
 mean: 0.71653, std: 0.00132, params: {'C': 0.10000000000000001},
 mean: 0.71653, std: 0.00132, params: {'C': 1.0},
 mean: 0.71653, std: 0.00132, params: {'C': 10.0},
 mean: 0.71653, std: 0.00132, params: {'C': 100.0},
 mean: 0.71653, std: 0.00132, params: {'C': 1000.0},
 mean: 0.71653, std: 0.00132, params: {'C': 10000.0},
 mean: 0.71653, std: 0.00132, params: {'C': 100000.0},
 mean: 0.71653, std: 0.00132, params: {'C': 1000000.0},
 mean: 0.71653, std: 0.0013

In [12]:
start_time = datetime.datetime.now()
print 'Training for the best C parameter = 0.01'
scores = cross_val_score(clfLog, X_nocat, y, scoring='roc_auc', cv=kf)
print 'Time elapsed:', datetime.datetime.now() - start_time, 'Score = ', np.mean(scores)

Training for the best C parameter = 0.01
Time elapsed: 0:00:14.696000 Score =  0.716559386445


Видим, что после исключения категориальных признаков модель показала незначительное улучшение.

In [13]:
categor_cols.remove('lobby_type')
##Уникальных идентификаторов героев:
np.unique(X[categor_cols])


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

В нашей выборке 108 разных героев, но мы видим, что макс. идентификатор героя = 112, т.е. всего героев, как минимум, 112.

In [14]:
N = 112
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in xrange(5):
        X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [15]:
X_nocat = np.hstack((X_nocat,X_pick))

In [16]:
clfLog, gsscores = bestC(clfLog, X_nocat, y)
gsscores

[mean: 0.60809, std: 0.00794, params: {'C': 1e-10},
 mean: 0.68931, std: 0.00128, params: {'C': 1.0000000000000001e-09},
 mean: 0.68933, std: 0.00128, params: {'C': 1e-08},
 mean: 0.68944, std: 0.00128, params: {'C': 9.9999999999999995e-08},
 mean: 0.69055, std: 0.00128, params: {'C': 9.9999999999999995e-07},
 mean: 0.69922, std: 0.00126, params: {'C': 1.0000000000000001e-05},
 mean: 0.72507, std: 0.00125, params: {'C': 0.0001},
 mean: 0.74633, std: 0.00173, params: {'C': 0.001},
 mean: 0.75174, std: 0.00227, params: {'C': 0.01},
 mean: 0.75195, std: 0.00245, params: {'C': 0.10000000000000001},
 mean: 0.75193, std: 0.00249, params: {'C': 1.0},
 mean: 0.75193, std: 0.00249, params: {'C': 10.0},
 mean: 0.75192, std: 0.00249, params: {'C': 100.0},
 mean: 0.75192, std: 0.00249, params: {'C': 1000.0},
 mean: 0.75192, std: 0.00249, params: {'C': 10000.0},
 mean: 0.75192, std: 0.00249, params: {'C': 100000.0},
 mean: 0.75192, std: 0.00249, params: {'C': 1000000.0},
 mean: 0.75192, std: 0.0024

In [17]:
clfLog

LogisticRegression(C=0.10000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

Т.к. логистическая регрессия с "мешком героев" - наш лучший алгоритм, запустим его на тестовой выборке:

In [23]:
#Готовим тестовые данные:
features_test = pandas.read_csv('./data/features_test.csv', index_col='match_id')
features_test.fillna(0, inplace=True)

categor_cols =  []
for c in features_test.columns:
    if c.find('_hero')>0:
        categor_cols.append(c)
categor_cols.append('lobby_type')

X_test = features_test.drop(categor_cols, axis=1)
X_test = StandardScaler().fit_transform(X_test)

In [26]:
#Заменяем категориальные признаки на мешок героев:
N = 112
X_pick = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(features_test.index):
    for p in xrange(5):
        X_pick[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_test = np.hstack((X_test,X_pick))        

In [30]:
#Делаем прогноз:
pred = clfLog.predict_proba(X_test)

In [33]:
#Максимальное значение целевой переменной:
np.max(pred[:, 1])

0.99645896971338632

In [34]:
#Минимальное значение целевой переменной:
np.min(pred[:, 1])

0.0085818482993009759

### Отчет по логистической регрессии

*Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?*

Ответ: Логистическая регрессия над всеми признаками дала результат 0.716550270785, что выше результата для градиентного бустинга на 100 деревьях и меньше. Это можно объяснить тем, что выборка, видимо, достаточно хорошо разделяется линейно, а также (в меньшей степени) тем, что мы не использовали StandardScaler для градиентного бустинга. Логистическая регрессия работает быстрее градиентного бустинга.

*Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?*

Ответ: При удалении категориальных признаков получили небольшое улучшение (новое значение метрики: 0.716559386445). Можно предположить, что логистическая регрессия либо плохо справляется с категориальными признаками в качестве предикторов, либо эти признаки не имели веса и только добавляли шум в модель.

*Сколько различных идентификаторов героев существует в данной игре?*

Ответ: Уникальных идентификаторов в выборке 108, но т.к. максимальный из них равен 112, можно предположить, что всего героев, как минимум, 112.

*Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?*

Ответ: Получили значительное улучшение результата модели (0.75195). Т.е. какие герои участвуют в матче - важный признак, который до добавления "мешка слов" ухудшал модель, т.к. логистическая регрессия плохо обрабатывает категориальные признаки (пока мы их не бинаризуем).

*Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?*

Ответ: максимальное - 0.99645896971338632, минимальное - 0.0085818482993009759